In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!ls "/content/gdrive/My Drive/CTW_2019"

 AE_Model_Conv2.h5		    data2019.h5
 AE_Model_Conv.h5		    data_feat.h5
 AutoEncoderModel1.h5		    ensemble_NDR_CNN_SNR_finetune.h5
 AutoEncoderModel2.h5		    ensemble_NDR_CNN_SNR.h5
 AutoEncoderModel.h5		    ensemble_PCA_CNN_SNR_2.h5
 best_AE1.h5			    ensemble_PCA_CNN_SNR_3.h5
 best_AE2.h5			    ensemble_PCA_CNN_SNR.h5
 best_AE3.h5			    h_Estimated_CTW_Train.h5
 best_AE4.h5			    Histogram_MSE.png
 best_cnn.h5			    Histogram.png
 best_cnn_ndr2.h5		    Histogram_RMSE.png
 best_cnn_ndr3.h5		    NDR.ipynb
 best_cnn_ndr4.h5		   'PCA(1056)+CNN'
 best_cnn_ndr5_adam.h5		   'PCA(66)+CNN'
 best_cnn_ndr.h5		    PCA+MLP_Hist.png
 best.h5			    PCA+MLP.ipynb
 Best_PCA_CNN_2.h5		    PCA_Unsupervised.sav
 Best_PCA_CNN.h5		    r_Position_CTW_Train.h5
 best_PCA.h5			    saved_model.pb
'Copy of AE.ipynb'		    SimpleAE.h5
'Copy of PCA+CNN'		    SimpleAE_pred.h5
 CTW2019_Dataset_h5.zip		    SNR_CTW_Train.h5
 CTW2019_logarithimic_mean_sqr.h5   test.out
 CTW2019_mean_sqr2.h5		    trrs_trial.h5
 CTW2019

In [9]:
import numpy as np # linear algebra
import keras
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt
import tensorflow as tf

In [10]:
'''#getting and processing data
#h_Estimated   ---> shape = (2, 924, 16, 17486)
#r_postition ---> shape = (3, 17486)
#SNR_Est    ---> shape = (16, 17486)


batch_size = 30
epochs = 50
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cnn_training.h5'
'''

"#getting and processing data\n#h_Estimated   ---> shape = (2, 924, 16, 17486)\n#r_postition ---> shape = (3, 17486)\n#SNR_Est    ---> shape = (16, 17486)\n\n\nbatch_size = 30\nepochs = 50\ndata_augmentation = False\nsave_dir = os.path.join(os.getcwd(), 'saved_models')\nmodel_name = 'keras_cnn_training.h5'\n"

In [13]:
f = h5py.File('/content/gdrive/My Drive/CTW_2019/h_Estimated_CTW_Train.h5','r')
H = f['h_Estimated']
f = h5py.File('/content/gdrive/My Drive/CTW_2019/r_Position_CTW_Train.h5','r')
pos = f['r_Position']



In [14]:
#Extracting just 3000 samples in a small area for testing purposes.
#Skip running this cell if not needed since it will replace all the variables needed ahead.
idx = np.argwhere(pos[0,:] < 3.0)
Y_comp = pos[:,idx]  
H = H[:,:,:,idx]

print(Y_comp.shape)
print(H.shape)

(3, 3050)
(2, 924, 16, 3050)


In [15]:
print(H.shape)
pos = Y_comp
H = np.array(H)
pos = np.array(pos)

del Y_comp, idx

(2, 924, 16, 3050)


In [ ]:
data = H[:,:,:,:1000]
print(data.shape)
xyz = pos[:,:1000]
print(xyz.shape)

(2, 924, 16, 1000)
(3, 1000)


In [ ]:
H = data[:,:,:,:900]
H = H.swapaxes(0, -1)
pos = xyz[:,:900]

H_test = data[:,:,:,900:]
H_test = H_test.swapaxes(0, -1)
pos_test = xyz[:,900:]

print(H.shape)
print(pos_test.shape)

(900, 924, 16, 2)
(3, 100)


In [16]:
from sklearn.model_selection import train_test_split
indices = list(range(0,pos.shape[1]))
train_idx, test_idx = train_test_split(indices, test_size=0.21, random_state=42)

X_train , X_test= H[:,:,:,train_idx], H[:,:,:,test_idx]
y_train , y_test= pos[:,train_idx], pos[:,test_idx]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
del H, pos

(2, 924, 16, 2409) (2, 924, 16, 641)
(3, 2409) (3, 641)


In [17]:
X_train = np.swapaxes(X_train, 0,3)
X_test = np.swapaxes(X_test, 0,3)
print(X_train.shape, X_test.shape)

(2409, 924, 16, 2) (641, 924, 16, 2)


In [ ]:
'''Both Single CFR'''
H1 = H[1]
H2 = H_test[1]

print(H2.shape)

Yd1 = np.sum(((H1*H1)[:,:,0]) + ((H1*H1)[:,:,1]), axis=0)
Yd2 = np.sum((H2*H2)[:,:,0] + (H2*H2)[:,:,1] , axis=0)
print(Yd2.shape)   #We have 16 links

H11 = H1[:,:,0] + H1[:,:,1] * 1j
H22 = H2[:,:,0] - H2[:,:,1] * 1j
G = H11 * H22
print(G.shape)


Nser = 1024
Nu = 924
G = np.concatenate((G, np.zeros((Nser-Nu, 16))), axis=0)

g = np.fft.fftn(G, axes=(0,))
print(g.shape)

phi_d = np.ndarray.max((np.square(np.abs(g))), axis=0)
phi_d = np.divide(phi_d,np.multiply(Yd1, Yd2))
print(phi_d.shape)

w_d = np.sqrt(np.multiply(Yd1,Yd2))/(np.sqrt(np.sum(Yd1)) * np.sum(Yd2))
print(w_d.shape)

TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)))
TRRS = TRRS**2

print(TRRS)

(924, 16, 2)
(16,)
(924, 16)
(1024, 16)
(16,)
(16,)
0.02238512572668457


In [18]:
'''H1 grid, H2 single CFR'''
#X = H

def trrs(H1, H2):
  Yd1 = np.sum((H1*H1)[:,:,:,0] + (H1*H1)[:,:,:,1] , axis=1)
  Yd2 = np.sum((H2*H2)[:,:,0] + (H2*H2)[:,:,1] , axis=0)
  #print((Yd1*Yd2).shape)

  H11 = H1[:,:,:,0] + H1[:,:,:,1] * 1j
  H22 = H2[:,:,0] - H2[:,:,1] * 1j
  G = H11 * H22
  
  Nser = 1024
  Nu = 924
  G = np.concatenate((G, np.zeros((H1.shape[0],Nser-Nu, 16))), axis=1)
  #print(G.shape)

  g = np.fft.fftn(G, axes=(1,))
  #print(g.shape)

  phi_d = np.ndarray.max((np.square(np.abs(g))), axis=1)
  phi_d = np.divide(phi_d,(Yd1*Yd2))
  #print(phi_d.shape)

  w_d = np.sqrt((Yd1 * Yd2))
  w_d = np.transpose(w_d)/(np.sqrt(np.sum(Yd1, axis=1)) * np.sum(Yd2))
  w_d = np.transpose(w_d)
  #print(w_d.shape)
  
  TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)), axis=1)
  TRRS = np.square(TRRS)

  idx = np.argmax(TRRS)

  return y_train[:,idx]


print(trrs(X_train,X_test[2]))
print(y_test[:,2])


[ 2.1672 -0.039  -0.5241]
[ 2.1672 -0.039  -0.5241]


In [ ]:
'''Complete train and test set'''
ans_grid = np.zeros((pos_test.shape))

for i in range(0,100):
  for j in range(0,3):
    ans_grid[j][i] = trrs(X,H_test[i])[j]

err = np.sqrt(((ans_grid - pos_test) ** 2).mean())

print(err)


0.8966715157365786


In [ ]:
#new_test
import time
start_time = time.time()
errors = []
for idx, ele in enumerate(X_test):
  sample_time = time.time()
  print(idx, ele.shape)
  errors.append(np.sqrt(np.sum((y_test[:,idx] - trrs(X_train, ele))**2)))
  print(errors[idx])
  print("------%s seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>" % (time.time()-sample_time))
print(np.mean(errors))
print("--- %s seconds ---" % (time.time() - start_time))


0 (924, 16, 2)
0.3642159386957139
------4.620216608047485 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
1 (924, 16, 2)
0.0
------2.4784791469573975 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
2 (924, 16, 2)
0.0
------2.5089943408966064 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
3 (924, 16, 2)
0.0
------2.473444700241089 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
4 (924, 16, 2)
0.03241943861327647
------2.4724411964416504 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
5 (924, 16, 2)
0.001296148139681515
------2.4805707931518555 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
6 (924, 16, 2)
0.0
------2.4932525157928467 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
7 (924, 16, 2)
0.0
------2.4800548553466797 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
8 (924, 16, 2)
0.002893095228297963
------2.4763965606689453 seconds for this sample ------->>>>>>>>>>>>>>>>>>>>>>>>>
9 (924, 16, 2)
0.0
------2.4